In [35]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
from generate_conanfiles import *

repo_dir = os.path.abspath('')
src_dir = os.path.join(repo_dir, "src")
package_xmls = get_package_xml_files(src_dir)
make_abs = lambda x: os.path.abspath(os.path.join(src_dir, x))
package_dirs = list(map(lambda x: Path(make_abs(x)).parent, package_xmls))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[PosixPath('/home/tbitz/repos/ros2conan/src/eclipse-cyclonedds/cyclonedds'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visualization/rqt_graph'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visualization/rqt_publisher'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visualization/rqt_shell'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visualization/tango_icons_vendor'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visualization/rqt_console'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visualization/rqt_service_caller'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visualization/rqt/rqt_gui_cpp'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visualization/rqt/rqt_gui_py'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visualization/rqt/rqt_py_common'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visualization/rqt/rqt'), PosixPath('/home/tbitz/repos/ros2conan/src/ros-visuali

In [37]:
import pandas as pd
from generate_conanfiles import has_cmakelists, has_setup_py

def create_dir_meta(dirname):
    return {
        "dir" : dirname,
        "cmake" : has_cmakelists(dirname),
        "python" : has_setup_py(dirname),
    }

def package_name(dirname):
    return os.path.split(dirname)[-1]

dir_meta = { package_name(dirname) : create_dir_meta(dirname) for dirname in package_dirs}

df = pd.DataFrame(dir_meta).transpose()
df.to_excel("package_metas.xlsx")
df

,dir,cmake,python
cyclonedds,/home/tbitz/repos/ros2conan/src/eclipse-cyclon...,True,False
rqt_graph,/home/tbitz/repos/ros2conan/src/ros-visualizat...,False,True
rqt_publisher,/home/tbitz/repos/ros2conan/src/ros-visualizat...,False,True
rqt_shell,/home/tbitz/repos/ros2conan/src/ros-visualizat...,False,True
tango_icons_vendor,/home/tbitz/repos/ros2conan/src/ros-visualizat...,True,False
...,...,...,...
iceoryx_binding_c,/home/tbitz/repos/ros2conan/src/eclipse-iceory...,True,False
iceoryx_hoofs,/home/tbitz/repos/ros2conan/src/eclipse-iceory...,True,False
map_msgs,/home/tbitz/repos/ros2conan/src/ros-planning/n...,True,False
keyboard_handler,/home/tbitz/repos/ros2conan/src/ros-tooling/ke...,True,False


In [38]:
build_system_unsure = [] 
both_cmake_and_python= [];
for pkg, meta in dir_meta.items():
    if (not meta['cmake'] and not meta['python']):
        build_system_unsure.append(pkg)
    if (meta['cmake'] and meta['python']):
        both_cmake_and_python.append(pkg)

print(f"Both cmake and python: {both_cmake_and_python}")
print(f"Neither cmake or python: {build_system_unsure}")

Both cmake and python: 8
Neither cmake or python: 0
